In [1]:
# creating paths to src and data folders in the repo
import sys
import pathlib
src_path = pathlib.Path().absolute().parent.parent / "src"
data_path = pathlib.Path().absolute().parent.parent / "data"
dump_path = data_path / 'initial_dump'

# train test split paths
train_path = data_path / 'fsm_tts/train'
test_path = data_path / 'fsm_tts/test'

# add src path to sys.path so it is searched in import statements
sys.path.append(str(src_path))

# basic imports for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import modeling packages
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# modeling metrics
from sklearn.metrics import classification_report, confusion_matrix

changes to data generators:

    train batch size 128 -> 100
    
    test batch size 10 -> 49

In [3]:
train_generator = ImageDataGenerator().flow_from_directory(str(train_path),
                                                           target_size=(150, 150),
                                                           batch_size=100,
                                                           class_mode='categorical')
test_generator = ImageDataGenerator().flow_from_directory(str(test_path),
                                                          target_size=(150, 150),
                                                          batch_size=49,
                                                          class_mode='categorical')

Found 10000 images belonging to 6 classes.
Found 931 images belonging to 6 classes.


changes to model architecture:
    
    number of output nodes 7 -> 6
    
    1st conv2d layer num_filters 32 -> 20
    
    num conv2d layers 2 -> 1

Trying to shorten train time while reducing the complexity of the features it picks up on. I suspect that the model will best identify the medium if it focuses more on the pixel to pixel changes on a short scale than large scale objects within the image.

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(20, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 20)      560       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 20)        0         
_________________________________________________________________
flatten (Flatten)            (None, 109520)            0         
_________________________________________________________________
dense (Dense)                (None, 20)                2190420   
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 126       
Total params: 2,191,106
Trainable params: 2,191,106
Non-trainable params: 0
_________________________________________________________________


changes to fit:
    
    validation_steps 7 -> 19
    
    epochs 5 -> 3

In [6]:
model.fit(
        train_generator,
        steps_per_epoch=61,
        epochs=3,
        validation_data=test_generator,
        validation_steps=19)

Epoch 1/3
 3/61 [>.............................] - ETA: 7:29 - loss: 857.0703 - accuracy: 0.2333

/opt/anaconda3/envs/capstone-env/lib/python3.6/site-packages/PIL/Image.py:2837: DecompressionBombWarning: Image size (108753612 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


 6/61 [=>............................] - ETA: 9:22 - loss: 828.2321 - accuracy: 0.3117

/opt/anaconda3/envs/capstone-env/lib/python3.6/site-packages/PIL/Image.py:2837: DecompressionBombWarning: Image size (114336000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


14/61 [=====>........................] - ETA: 9:24 - loss: 439.9511 - accuracy: 0.2993

...

59/61 [============================>.] - ETA: 22s - loss: 108.1938 - accuracy: 0.2124

/opt/anaconda3/envs/capstone-env/lib/python3.6/site-packages/PIL/Image.py:2837: DecompressionBombWarning: Image size (89505558 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


61/61 [==============================] - 794s 13s/step - loss: 104.7047 - accuracy: 0.2120 - val_loss: 1.8157 - val_accuracy: 0.1504
Epoch 2/3
58/61 [===========================>..] - ETA: 32s - loss: 1.7752 - accuracy: 0.3526

/opt/anaconda3/envs/capstone-env/lib/python3.6/site-packages/PIL/Image.py:2837: DecompressionBombWarning: Image size (145130209 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


61/61 [==============================] - 770s 13s/step - loss: 1.7738 - accuracy: 0.3556 - val_loss: 1.8908 - val_accuracy: 0.4425
Epoch 3/3
61/61 [==============================] - 775s 13s/step - loss: 1.7456 - accuracy: 0.3603 - val_loss: 1.7325 - val_accuracy: 0.4458


In [7]:
confusion_matrix(test_generator.classes, np.argmax(model.predict(test_generator), axis=1))

array([[415,   0,   0,   0,   0,   0],
       [140,   0,   0,   0,   0,   0],
       [189,   0,   0,   0,   0,   0],
       [ 41,   0,   0,   0,   0,   0],
       [132,   0,   0,   0,   0,   0],
       [ 14,   0,   0,   0,   0,   0]])

welp, thats not great. looks like i need to do some more class balancing.